Initialisations to change the base directory of the entry scripts and to update mxnet to the newest mxnet-mkl

In [1]:
!sed -i 's/examples.//g' battlesnake_src/train.py
!sed -i 's/examples.//g' battlesnake_src/networks/agent.py
!sed -i '1i import subprocess\nsubprocess.run(["pip",  "uninstall", "mxnet-mkl", "-y"])\nsubprocess.run(["pip",  "install", "mxnet-mkl", "--pre"])\nsubprocess.run(["pip",  "install", "pandas"])' battlesnake_src/train.py

In [3]:
import boto3
import sagemaker
import numpy as np

from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.mxnet.estimator import MXNet

## Initialise sagemaker
We need to define several parameters prior to running the training job. 
note: `local_mode` defines whether to run the code within this notebook or to run a sagemaker training job

In [4]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()  
s3_output_path = 's3://{}/'.format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

# run in local_mode on this machine or as a SageMaker TrainingJob
local_mode = False

if local_mode:
    instance_type = 'local'
else:
    instance_type = "ml.c5.2xlarge"
role = sagemaker.get_execution_role()
print("Using IAM role arn: {}".format(role))
# only run from SageMaker notebook instance
if local_mode:
    !/bin/bash ./setup.sh
cpu_or_gpu = 'gpu' if instance_type.startswith('ml.p') else 'cpu'

S3 bucket path: s3://sagemaker-us-east-2-412868550678/
Using IAM role arn: arn:aws:iam::412868550678:role/service-role/AmazonSageMaker-ExecutionRole-20191112T133326


## Define the attributes of the training job
Use `job_name_prefix` to identify the sagemaker training job for this.

In [4]:
# create a descriptive job name 
job_name_prefix = 'Battlesnake-job'
max_jobs = 3
max_parallel_jobs = 3

## Define the metrics to evaluate your training job
The regex for this metric was defined based on what is printed in the training script `examples/train.py`

In [5]:
metric_definitions = [
    {'Name': 'timesteps', 'Regex': '.*Mean timesteps ([-+]?[0-9]*[.]?[0-9]+([eE][-+]?[0-9]+)?)'},
]

# Define the hyperparameters of your job

In [6]:
hyperparameter_ranges = {
    'buffer_size': IntegerParameter(1000, 6000),
    'update_every': IntegerParameter(10, 20),
    'batch_size': IntegerParameter(16, 256),

    'lr_start': ContinuousParameter(1e-5, 1e-3),
    'lr_factor': ContinuousParameter(0.5, 1.0),
    'lr_step': IntegerParameter(5000, 30000),
    
    'tau': ContinuousParameter(1e-4, 1e-3),
    'gamma': ContinuousParameter(0.85, 0.99),
    
    'depth': IntegerParameter(10, 256),
    'depthS': IntegerParameter(10, 256),
}

static_hyperparameters = {
    'qnetwork_type': "attention",
    'seed': 666,
    'number_of_snakes': 4,
    'episodes': 1000,
    'print_score_steps': 10,
    'activation_type': "softrelu",
    'state_type': 'one_versus_all',
    'sequence_length': 2,
    'repeat_size': 3,
    'kernel_size': 3,
    'starting_channels': 6,
    'map_size': "[15, 15]",
    'snake_representation': 'bordered-51s',
    'save_model_every': 700,
    'eps_start': 0.99,
    'models_to_save': 'local'
}

Defines the estimator. Firstly, try run this with `local_model = True` to test your entry_point script

In [7]:
estimator = MXNet(entry_point="train.py",
                  source_dir='battlesnake_src',
                  dependencies=["battlesnake_gym/"],
                  role=role,
                  train_instance_type=instance_type,
                  train_instance_count=1,
                  output_path=s3_output_path,
                  framework_version="1.6.0",
                  py_version='py3',
                  base_job_name=job_name_prefix,
                  metric_definitions=metric_definitions,
                  hyperparameters=static_hyperparameters
                 )
if local_mode:
    estimator.fit()

Start the Hyperparameter optimisation sagemaker jobs!

In [8]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name='timesteps',
                            objective_type='Maximize',
                            hyperparameter_ranges=hyperparameter_ranges,
                            metric_definitions=metric_definitions,
                            max_jobs=max_jobs,
                            max_parallel_jobs=max_parallel_jobs,
                            base_tuning_job_name=job_name_prefix)
tuner.fit()

Now wait for the hyper parameter turner to complete

# Collect the best model and create an endpoint
Obtain an s3 URL of the best model

In [1]:
best_training_job = tuner.best_training_job()
best_model_path = "{}/{}//output//model.tar.gz".format(s3_output_path, best_training_job)
print("Best model location {}".format(best_model_path))

NameError: name 'tuner' is not defined

In [5]:
from sagemaker.mxnet import MXNetModel
sagemaker_model = MXNetModel(model_data=best_model_path,
                             entry_point='predict.py',
                             role=role,
                             framework_version='1.6.0',
                             source_dir='battlesnake_src',
                             dependencies=["battlesnake_gym/"],
                             py_version='py3')
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type=instance_type,
                                   endpoint_name='battlesnake-endpoint')

-------------!

Testing the endpoint

In [7]:
data1 = np.zeros(shape=(1, 2, 3, 17, 17))
data2 = np.zeros(shape=(1, 2))
data3 = np.zeros(shape=(1, 2))
data4 = np.zeros(shape=(1, 2))
predictor.predict({"state": data1, "snake_id": data2, "turn_count": data3, "health": data4})

[[-0.09669163078069687,
  1.009218692779541,
  1.089157223701477,
  0.7409930229187012]]